# RAG, Chatbots and Agents

Concepts:
- Measuring the value of corporate philanthropy
- Retrieval Augmented Generation (RAG)
- Chatbots
- LLM Agents

References:
- Patrick Lewis, Ethan Perez, Aleksandra Piktus, Fabio Petroni, Vladimir Karpukhin, Naman Goyal†, Heinrich Küttler, Mike Lewis†, Wen-tau Yih, Tim Rocktäschel, Sebastian Riedel, Douwe Kiela, 2021, 
Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
- Lim, T., 2010, Measuring the value of corporate philanthropy: Social impact, business benefits, and investor returns. published by CECP.

In [1]:
# Chroma requires SQLite > 3.35, if you encounter issues with having too
# low of a SQLite version, try installing pysqlite3-binary, then enter
# the following three lines in sqlite_version.py to swap the packages:
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
import sqlite_version   # hack to use lower version of SQLite

# !ollama pull mxbai-embed-large  # vector embeddings model in Ollama
import ollama

from langchain_community.llms import Ollama
from langchain_text_splitters import RecursiveCharacterTextSplitter
# import pdfplumber    # PDF document loaders
# from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
import chromadb
from tqdm import tqdm
import textwrap

def wrap(text):
    """Helper to wrap text for pretty printing"""
    return "\n".join([textwrap.fill(s) for s in text.split('\n')])

## Retrieval Augmented Generation

Retrieval-Augmented Generation (RAG) combines two steps: retrieving information from a large database or knowledge base external to the LLM, and then generating a response using this retrieved information. By having access to up-to-date and relevant information, the LLM can provide answers that are more accurate and detailed.

### Hallucination

Hallucination is when an AI makes up information that sounds believable but is actually incorrect or nonsensical. With RAG, since the AI’s responses are based on relevant and factual information, there is a much higher chance it will stay truthful and reliable.


### Langchain APIs

LangChain is a framework designed to facilitate the creation of applications that utilize language models. It simplifies the process of integrating language models with external data sources and other AI tools, for example, by providing a common interface to models served both locally (e.g. Ollama) or remotely (e.g.cloud-based server).

https://python.langchain.com/v0.1/docs/integrations/llms/ollama/




In [2]:
# temperature set low but nonzero so that chat responses can vary slightly
llm = Ollama(model="llama3:instruct", temperature=0.3)

LangChain includes convenient loaders for input documents of many formats.

Documents are read and split into chunks of, say, around 1000 characters with overlap. To drive the demonstrations of a chatbot and a collaborative conversation between multiple LLM agents, we load and ground their responses in a textbook on measuring the value of corporate philanthropy. This subject is about the comprehensive approach that considers social impact, business benefits, and investor expectations, so that companies can better understand and optimize the effectiveness of their philanthropic initiatives.

In [3]:
loader = UnstructuredMarkdownLoader('MVCP.md')
document = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunked_documents = text_splitter.split_documents(document)
print('Number of chunks:', len(chunked_documents))
print(wrap(str(chunked_documents[:2])))

Number of chunks: 198
[Document(page_content='MEASURING THE VALUE OF CORPORATE PHILANTHROPY:
SOCIAL IMPACT, BUSINESS BENEFITS, AND INVESTOR RETURNS\nby\nTerence
Lim, Ph.D.\nReport Author and Manager, Standards and
Measurement,\nCommittee Encouraging Corporate Philanthropy\n(through
the 2008–2009 Goldman Sachs Public Service Program)\n\nHow to measure
the value and results of corporate philanthropy remains\none of
corporate giving professionals’ greatest challenges. Social
and\nbusiness benefits are often long-term or intangible, which
make\nsystematic measurement complex. And yet: Corporate philanthropy
faces\nincreasing pressures to show it is as strategic,cost-effective,
and value-enhancing\nas possible. The industry faces a critical need
to assess current practices and\nmeasurement trends, clarify the
demands practitioners face for impact evidence,\nand identify the most
promising steps forward in order to make progress on
these\nchallenges.', metadata={'source': 'MVCP.md'}),
Docume

### Vector Database

Chroma DB is an open-source vector store that is utilized for the storage and retrieval of vector embeddings. It performs fast search of text data, and is used by semantic search engines and large language models.


In [4]:
client = chromadb.Client()
#client.delete_collection(name="docs") 
collection = client.get_or_create_collection(name="docs")

Use Ollama to serve a word embeddings models, such as the "mxbai-embed-large" model, for convert and store in Chroma DB's vector database.

In [5]:
# store each document in a vector embedding database
for i, d in enumerate(chunked_documents):
    response = ollama.embeddings(model="mxbai-embed-large", prompt=d.page_content)
    embedding = response["embedding"]
    collection.add(
        ids=[str(i)],
        embeddings=[embedding],
        documents=[d.page_content]
    )

Write a prompt query

In [23]:
# a question-answer prompt
query = "Why is it important to measure the value of corporate philanthropy?"

Embed the prompt query

In [24]:
# generate an embedding for the query
response = ollama.embeddings(
  prompt=query,
  model="mxbai-embed-large"
)
len(response['embedding'])   # vector length

1024

Retrieve the 3 most similar document chunks from the vector database

In [25]:
# retrieve the most relevant doc
results = collection.query(
  query_embeddings=[response["embedding"]],
    n_results=3,
)
data = ". ".join(results['documents'][0])
print(textwrap.fill(data))

A model for measuring the influence of corporate philanthropic
initiatives on employee engagement.. consumer satisfaction and
companies philanthropic record only when companies are perceived to
have strong product quality and innovation capabilities and or operate
in consumer oriented industries 22 Designing a measurement framework
for loyalty should begin with an assessment of the perceptions
customers have already developed as a result of a company s corporate
philanthropic initiatives and whether these perceptions are
contributing to higher loyalty scores Figure 9 suggests such a
framework based on the literature reviewed Figure 9 A Framework for
Measuring Customer Loyalty and Corporate Philanthropy Source Adapted
from Bhatt acharya C B Sen S 2004 and Smith V Langford P 2009 A
company s marketing department is likely already to have implemented
its own customer loyalty metrics in which case it is sensible to
leverage these along with customized deliberate customer research It
is imp

Generate a response combining the query and data we retrieved in previous step

In [26]:
prompt = f"""
Only use relevant information in the following text delimited by triple quotes:

'''{data}.''' 

Respond to this prompt: {query}.

In your response, only use information in the given text.
Do not mention that you referred to the text."""

output = llm.invoke(prompt)
print(textwrap.fill(output))

It is imperative that the additional factors affecting loyalty scores,
such as philanthropic strategies and disclosures, are linked to can be
successful in financial value. This suggests that measuring the value
of corporate philanthropy is important because it can create long-term
business value. Additionally, a high-quality measurement process
demonstrates that a company recognizes its philanthropic strategies
can be successful in creating long-term business value, which can lead
to differentiating themselves to the investor community through their
disclosures about philanthropic efforts.


In [27]:
# show the supporting source document chunk ids
print(set(results['ids'][0]))

{'95', '120', '187'}


## Chatbot

The chatbot is an LLM instance, where a record of the past conversation is included in the next prompt, so that the chatbot can reply to the user's question with memory of past queries and responses.


In [11]:
system_prompt = "You are a competent AI business manager."

In [12]:
instruction = """
I will provide you with ideas.
Please summarize and briefly explain all 
the ideas, including those I have provided you earlier, in a bulleted list.
"""

In [1]:
# Simulate a sequence of user queries, which expect the LLM to remember past queries and responses.
user_queries = [
    "Idea: Sum up the amount of charitable contributions.",
    "Idea: Measure the dollar value of goods contributed.",
    "Idea: Count total employee volunteering hours.",
]

Concatenate past query and response turns to form next prompt


In [14]:
# Loop over number of incremental user queries and build up memory of responses

# These special input symbols and sequences are needed to construct the prompt
# including the past conversation. See
# https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/
tokens = {'bos_token': '<|begin_of_text|>',
          'eos_token': '<|end_of_text|>',
          'pad_token': '<|reserved_special_token_250|>',
          'system_header': '<|start_header_id|>system<|end_header_id|>',
          'assistant_header': '<|start_header_id|>assistant<|end_header_id|>',
          'user_header': '<|start_header_id|>user<|end_header_id|>',
          'eot': '<|eot_id|>', 
}

responses = []   # memory of past AI-assistant turns
for turn in tqdm(range(len(user_queries))):
    # 1. Start with system message 
    prompt = tokens['bos_token']
    prompt += "\n".join([tokens['system_header'],
                         system_prompt,
                         tokens['eot']]) + '\n\n'

    # 2. Add user instruction and first user prompt
    prompt += "\n".join([tokens['user_header'],
                         instruction,
                         user_queries[0],
                         tokens['eot']]) + '\n\n'

    # 3. Loop through past turns of AI response and user prompt
    for response, query in zip(responses, user_queries[1:][:len(responses)]):
        
        # a. Add AI response from memory
        prompt += "\n".join([tokens['assistant_header'],
                             response,
                             tokens['eot']]) + '\n\n'
        
        # b. Add next user prompt
        prompt += "\n".join([tokens['user_header'],
                             query,
                             tokens['eot']]) + '\n\n'

    # 4. End with assistant header to ellicit AI response
    prompt += tokens['assistant_header']

    response = llm.invoke(prompt)
    responses.append(response)

100%|██████████| 3/3 [00:16<00:00,  5.53s/it]


Show the final response from the last turn of the Chatbot

In [15]:
print('FINAL RESPONSE:')
print(wrap(response))

FINAL RESPONSE:
Here's the updated list with the new idea:

* **Sum up the amount of charitable contributions**: This idea likely
refers to calculating and aggregating the total amount of donations
made by a company or organization to various charities. This could be
an important metric for tracking philanthropic efforts and measuring
social responsibility.
* **Measure the dollar value of goods contributed**: This idea
suggests quantifying the monetary value of goods, products, or
services donated by a company or individual to charitable causes. This
could include in-kind donations, product drives, or other forms of
non-monetary giving.
* **Count total employee volunteering hours**: This idea focuses on
tracking and aggregating the total number of hours that employees
spend volunteering for charitable organizations or community events.
This metric can help measure employee engagement, corporate social
responsibility, and the impact of company-sponsored volunteer
programs.

Please provi

Show the last prompt that was asked of the chatbot: it includes the entire memory of the past conversation.

In [16]:
print("FINAL PROMPT:")
print(prompt)

FINAL PROMPT:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a competent AI business manager.
<|eot_id|>

<|start_header_id|>user<|end_header_id|>

I will provide you with ideas.
Please summarize and briefly explain all 
the ideas, including those I have provided you earlier, in a bulleted list.

Idea: Sum up the amount of charitable contributions.
<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
Since this is the first idea you've provided, I'll start by summarizing it:

* **Sum up the amount of charitable contributions**: This idea likely refers to calculating and aggregating the total amount of donations made by a company or organization to various charities. This could be an important metric for tracking philanthropic efforts and measuring social responsibility.

Please go ahead and provide more ideas, and I'll add them to the list!
<|eot_id|>

<|start_header_id|>user<|end_header_id|>
Idea: Measure the dollar value of goods contributed.
<|eot_id|>

<|s

## Agents

In multi-agent LLM systems, multiple LLM agents interact with each other or work in collaboration to achieve complex tasks or goals. Frameworks such as Microsoft's Autogen provides a multi-agent conversation framework as a high-level abstraction with an open-source library to build complex workflows.

- https://www.microsoft.com/en-us/research/project/autogen/

We demonstrate an illustrative workflow of a conversation between three LLM agents, who perform the roles of a Chief Executive Officer, a Chief Giving Officer (who manages the company's corporate philanthropy program), and an AI Assistant. Over several turns, they incrementally brainstorm and build up a plan for measuring the value of their program, drawing only from information in the RAG document store.

### Role-playing

__AI Assistant agent__

This agent responds to a query using RAG.  To reduce hallucination, the prompt is clearly written to stick to the retrieved text, and to simply respond "I don't know" if the answer cannot be found in the text.  This agent performs the role of an AI Corporate Giving expert assistant (named CorGi), whose responses only use information retrieved from the textbook stored in the underlying database.

In [17]:
def rag_agent(text, n_results=3):
    """Role of a RAG-based question-answering agent"""
    response = ollama.embeddings(prompt=text, model="mxbai-embed-large")

    results = collection.query(query_embeddings=[response["embedding"]],
                               n_results=n_results)
    data = "\n".join(results['documents'][0])

    rag_prompt=f"""
You are a helpful AI assistant.
Only use the information in the following text delimited by triple quotes:

'''{data}.''' 

Provide an answer to this question: {text}.

You answer must only contain information in the given text.
If the information requested is not in the text, say "I don't know".
Be concise.
Do not mention that you referred to the text."""
    
    # generate a response combining the prompt and data
    output = llm.invoke(rag_prompt)
    return output, results['ids'][0]

__Chief Giving Officer agent__

This role collects and summarizes of the responses received from the AI Assistant agent, to present to the Chief Executive Officer.

In [18]:
def cgo_agent(text):
    """Role of an information-summarizing Chief Giving Officer"""
    cgo_prompt = f"""
You are the chief giving officer of a company. 
You want to describe a plan for measuring the value
of your company's corporate philanthropy program. 
Only use the most relevant information about the plan 
given in the following text delimited by triple quotes:

'''{text}.'''

Please summarize and explain all the parts of the plan given in the text using a bulleted list.
Be complete and concise.
In your summary, do not use any information that is not in the given text.
"""
    output = llm.invoke(cgo_prompt)
    return output

__Chief Executive Officer agent__

This LLM agent generates questions, given the text of the plan, for the other agents to answer and add improvements.

In [19]:
def ceo_agent(text):
    """Role as an inquisitive Chief Executive Officer"""
    ceo_prompt = f"""
You are the chief executive officer of a company. 
You want to know about the plan for measuring the value
of your company's corporate philanthropy program. 
The plan is given in the text delimited by triple quotes:

'''{text}.'''

Please ask a simple, general question for an idea to improve the your company's plan
that is different than the text.
Be concise."""

    output = llm.invoke(ceo_prompt)
    return output

Loop over several turns of the conversation between the three agents, who collaboratively develop the plan.

In [20]:
memory = '* Measure the amount of monetary donations.'
docs = []
for turn in range(10):
    print(f"Turn {turn+1}...")
    question = ceo_agent(memory)
    print('CEO >>>', wrap(question))
    print('\n------------------------------\n')
    answer, doc = rag_agent(question)
    docs.extend(doc)
    print('Corgi-AI >>>', wrap(answer))
    print('\n------------------------------\n')
    memory = cgo_agent("\n".join([memory, answer]))
    print('CGO >>>', wrap(memory))
    print('\n------------------------------')
print('Source documents:', set(docs))

Turn 1...
CEO >>> What metrics would you recommend tracking to gauge the program's
impact on employee engagement and morale?

------------------------------

Corgi-AI >>> How are outcome metrics designed and tracked? Draws from knowledge and
experience of third-party domain-area experts engaged to collect
(and/or supervise the collection of) data and then to conduct
evaluation analysis.

Outcome metrics for gauging program's impact on employee engagement
and morale: How are outcome metrics designed and tracked?

------------------------------

CGO >>> Here is a summary of the plan for measuring the value of the company's
corporate philanthropy program:

• **Measure monetary donations**: The plan involves tracking the
amount of monetary donations made by the company.

• **Outcome metrics designed and tracked with expert input**: To
design and track outcome metrics, the company will draw from the
knowledge and experience of third-party domain-area experts who will
collect (and/or supervi

Show the final turn

In [21]:
print("FINAL ANSWER")
print(wrap(memory))

FINAL ANSWER
Here is a summary of the plan for measuring the value of the company's
corporate philanthropy program:

• **Monetary donations**: Track the amount of monetary donations made
by the company to measure the financial aspect of the program.

• **Outcome metrics**: Design and track outcome metrics with expert
input to measure the impact of the corporate philanthropy program.
This includes metrics for:
        + Program impact on employee engagement and morale
        + Customer loyalty and perceptions

• **Existing customer loyalty metrics**: Use existing customer loyalty
metrics to communicate the value of the corporate philanthropy program
to stakeholders, including customers.

• **Deliberate customer research**: Conduct research specifically
designed to understand how customers perceive the company's corporate
philanthropic initiatives and whether these perceptions contribute to
higher loyalty scores.

• **Perceptions and loyalty scores**: Measure the perceptions
customers h

For attribution, show the source reference documents which supported the responses. 

In [22]:
print("Source documents reference id's:", set(docs))

Source documents reference id's: {'15', '194', '5', '98', '120', '87', '2', '1', '102', '109', '94', '8', '97', '4', '85', '86', '28', '66', '95'}
